In [31]:
# This notebook shows an example of creating a DeepLabCut dataset from a collection of images
# and labels in TPS format.
# See also the config.yaml file


In [ ]:
import glob
import shutil
from PIL import Image

In [ ]:
## images from a FA_MAT_GAT dataset
images = glob.glob(r"FA_MAT_GAT/**/**/*.tif")
images

In [71]:
## Copying the images to a DeepLabCut project as if they were extracted from a video, convert to png

for f in images:
    folder = f[f.find("\\")+1:f.rfind(("\\"))] 
    nf = folder.replace("\\", ".")
    img_name = f[f.rfind("\\")+1:]
    img_name = img_name.replace(".tif", ".png")
    new_name = "labeled-data//no_video//" + nf + "." + img_name                              
    img = Image.open(f)
    img.save(new_name, format="png")
    

In [ ]:
# join all the tps file into a csv for use in DeepLabCut

tps_files = glob.glob(r"FA_MAT_GAT/**/**/*.tps")

In [107]:
## this is the CSV header structure for DeepLabCut
## scorer bobdan, 15 bodyparts

line1 = "scorer,," + ",bobdan"*30
line2 = "individuals,," + ",individual1"*30
line3 = "bodyparts,,,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12,13,13,14,14,15,15"
line4 = "coords,,,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y,x,y"

csv = line1 + "\n" + line2 + "\n" + line3 + "\n" + line4 + "\n"

## And individual rows for each image, converting from TPS format to coords for deeplabcut

for f in tps_files:
    with open(f) as fp:
        lines = fp.readlines()
        for i in range(len(lines)):
            line = lines[i]
            if line.split("=")[0] == 'LM': 
                lm = int(line.split("=")[1])
                coords = []
                for k in range(lm):
                    i +=1
                    line = lines[i]
                    x, y = map(float, line.split(" "))
                    y = 1920 - y
                    coords.append(x)
                    coords.append(y)
                i += 1
                img, img_name = lines[i].split("=")
                i= i+ 2                
                folder = f[f.find("\\")+1:f.rfind(("\\"))] 
                nf = folder.replace("\\", ".")
                img_name = img_name.rstrip("\n")
                img_name = img_name.replace(".tif", ".png")
                new_name = nf + "." + img_name                              
                row = "labeled-data,no_video," + new_name + "," + ",".join(str(int(c)) for c in coords)
                csv += row + "\n"


In [108]:
with open(r"labeled-data/no_video/CollectedData_bobdan.csv", "wt") as f:
    f.write(csv)

In [ ]:
import pandas as pd
df2 = pd.read_csv("./labeled-data/no_video/CollectedData_bobdan.csv", header=[0,1,2,3], index_col=[0,1,2])
df2

In [114]:
df2.to_hdf("./labeled-data/no_video/CollectedData_roberto.h5", "df_with_missing",
    format="fixed",
    mode="w")